In [2]:
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch
from pathlib import Path
import pandas as pd

In [3]:
PATH = Path("/home/shaun/data/kaggle/iwild/")

In [4]:
[print(i) for i in PATH.iterdir()]

/home/shaun/data/kaggle/iwild/sample_submission.csv
/home/shaun/data/kaggle/iwild/train.csv
/home/shaun/data/kaggle/iwild/animal_preds_train_res50-1.npy
/home/shaun/data/kaggle/iwild/is_animal_train-res50.npy


[None, None, None, None]

## Metrics

In [5]:
def accuracy(out, labels):
    preds = torch.argmax(out, dim=1)
    return (preds == labels).float().mean()

## Data prep

In [7]:
is_animal = np.load(PATH/"is_animal_train-res50.npy")

In [38]:
is_animal.shape

(196299, 2)

In [9]:
animal = np.load(PATH/"animal_preds_train_res50-1.npy")

In [39]:
animal.shape

(196299, 13)

In [11]:
joined = np.concatenate((is_animal, animal), axis=1)

In [12]:
df = pd.read_csv(PATH/"train.csv")
classes = df["category_id"].unique()
classes_map = {classes[i]:i for i in range(len(classes))}
new_labels = np.array([ classes_map[i] for i in df["category_id"]])

In [40]:
classes_map

{0: 1,
 1: 6,
 3: 2,
 4: 4,
 8: 3,
 10: 12,
 11: 7,
 13: 5,
 14: 10,
 16: 8,
 17: 9,
 18: 11,
 19: 0,
 22: 13}

In [35]:
len(new_labels)

196299

In [13]:
x = joined
y = new_labels

In [14]:
split=200
x_train, y_train, x_valid, y_valid = map(torch.tensor, (x[split:], y[split:], x[:split], y[:split])) 

In [15]:
train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

In [23]:
bs = 1000
train_data_loader = DataLoader(train_dataset, batch_size=bs)
valid_data_lodaer = DataLoader(valid_dataset, batch_size=bs)

## Model definition

In [30]:
in_features = 15
num_classes = 14

model = nn.Sequential(
    nn.Linear(in_features, 30),
    nn.ReLU(),
    nn.Linear(30, num_classes)
)

In [25]:
model

Sequential(
  (0): Linear(in_features=15, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=14, bias=True)
)

## Train Definition

In [31]:
lr = 1e-3

ce_loss = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr=lr)

In [32]:
def step(batch, train):
    data, labels = batch #get batch
    out = model(data.float()) # get predictions
    loss = ce_loss(out, labels) # calcualte loss
    acc = accuracy(out, labels)
    if train:
        optim.zero_grad() # zero graidents
        loss.backward() # calcualte gradients
        optim.step() #update parameters

    return loss.item(), acc


In [33]:
def train_epoch(dataloader, train=True):
    loss_hist = []
    acc_hist = []
    for batch in dataloader:
        loss_t, acc_t = step(batch, train=train)
        loss_hist.append(loss_t)
        acc_hist.append(acc_t)
        
    return np.mean(loss_hist), np.mean(acc_hist)

In [34]:
epochs = 10
loss_train = np.empty(epochs)
acc_train = np.empty(epochs)
loss_eval = np.empty(epochs)
acc_eval = np.empty(epochs)

for ep  in range(epochs):
    print(f'Epoch: {ep}')
    loss_t, acc_t = train_epoch(train_data_loader)
    loss_v, acc_v = train_epoch(valid_data_lodaer, train=False)
    loss_train[ep], acc_train[ep], loss_eval[ep], acc_eval[ep] =  loss_t, acc_t, loss_v, acc_v
    print(f' Train Loss: {np.round(loss_t,3)}, Valid Loss:{np.round(loss_v,3)}')
    print(f' Train acc: {np.round(acc_t,3)}, Valid acc:{np.round(acc_v,3)}')


Epoch: 0
 Train Loss: 1.615, Valid Loss:0.727
 Train acc: 0.6579999923706055, Valid acc:0.7799999713897705
Epoch: 1
 Train Loss: 0.504, Valid Loss:0.348
 Train acc: 0.8880000114440918, Valid acc:0.9350000023841858
Epoch: 2
 Train Loss: 0.277, Valid Loss:0.224
 Train acc: 0.9409999847412109, Valid acc:0.949999988079071
Epoch: 3
 Train Loss: 0.213, Valid Loss:0.188
 Train acc: 0.9459999799728394, Valid acc:0.949999988079071
Epoch: 4
 Train Loss: 0.193, Valid Loss:0.173
 Train acc: 0.9470000267028809, Valid acc:0.949999988079071
Epoch: 5
 Train Loss: 0.186, Valid Loss:0.167
 Train acc: 0.9470000267028809, Valid acc:0.9549999833106995
Epoch: 6
 Train Loss: 0.182, Valid Loss:0.163
 Train acc: 0.9470000267028809, Valid acc:0.949999988079071
Epoch: 7
 Train Loss: 0.18, Valid Loss:0.161
 Train acc: 0.9470000267028809, Valid acc:0.949999988079071
Epoch: 8
 Train Loss: 0.179, Valid Loss:0.16
 Train acc: 0.9470000267028809, Valid acc:0.949999988079071
Epoch: 9
 Train Loss: 0.178, Valid Loss:0.159

## Submission